In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/Final IMG' -r '/content/'

In [ ]:
import os

import cv2
import imutils as imutils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # machine learning
from tqdm import tqdm # make your loops show a smart progress meter 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sn

RANDOM_SEED = 1
IMG_SIZE = (256, 256, 3) # size of vgg16 input
IMG_PATH = "/content/Final IMG/"

print(os.listdir(IMG_PATH))

['val', 'train', 'test']


In [ ]:
Test_Path = IMG_PATH + "test/"
Training_Path = IMG_PATH + "train/"
Validation_Path = IMG_PATH + "val/"

In [ ]:
def number_of_imgs(path):
    print(path)
    for value in os.listdir(path):
        print(value, "has", len(os.listdir(path + value)), "imgs")
    print('\n')
    
    
number_of_imgs(Test_Path)
number_of_imgs(Training_Path)
number_of_imgs(Validation_Path)

/content/Final IMG/test/
non caries has 22 imgs
caries has 94 imgs


/content/Final IMG/train/
non caries has 22 imgs
caries has 1038 imgs


/content/Final IMG/val/
non caries has 22 imgs
caries has 94 imgs




In [ ]:
!pip install rich

In [ ]:
from rich.progress import track
#track(os.listdir(Training_Path))

In [ ]:
X_train = []
y_train = []
X_val = []
y_val = []

import cv2


for files in track(os.listdir(Training_Path)):
  for file_name in os.listdir(Training_Path + '/' + files):
    img = cv2.imread(Training_Path + '/' + files + '/' + file_name)
    img = cv2.resize(img, (256, 256))
    X_train.append(img)
    y_train.append(files)


for files in os.listdir(Validation_Path):
  for file_name in track(os.listdir(Validation_Path + '/' + files)):
    img = cv2.imread(Validation_Path + '/' + files + '/' + file_name)
    img = cv2.resize(img, (256, 256))
    X_val.append(img)
    y_val.append(files)


Output()

Output()

Output()

In [ ]:
X_train= np.array(X_train)
X_val = np.array(X_val)

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
y_train = lb.fit_transform(y_train)
y_val = lb.transform(y_val)

In [ ]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.applications.vgg16 import VGG16
base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=IMG_SIZE)

base.trainable=True
model = Sequential()
model.add(base)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.layers import Flatten,Dense,Dropout

model.add(Flatten())
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               16777728  
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 31,493,442
Trainable params: 31,493,442
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)

In [ ]:
X_train.shape

(1060, 256, 256, 3)

In [ ]:
EPOCHS = 25

history = model.fit(
    x=X_train,
    y=y_train,
    steps_per_epoch=34,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    validation_steps=25,
    callbacks=[early_stopping]
)

print("Training Done")

Epoch 1/25
34/34 [==============================] - 17s 506ms/step - loss: 0.1111 - accuracy: 0.9736
Epoch 2/25
34/34 [==============================] - 17s 514ms/step - loss: 0.0806 - accuracy: 0.9792
Epoch 3/25
34/34 [==============================] - 17s 505ms/step - loss: 0.0742 - accuracy: 0.9764
Epoch 4/25
34/34 [==============================] - 17s 506ms/step - loss: 0.0713 - accuracy: 0.9764
Epoch 5/25
34/34 [==============================] - 17s 507ms/step - loss: 0.0576 - accuracy: 0.9774
Training Done


In [ ]:
model.save("/content/drive/MyDrive/MODELS/VGG16.h5")

In [ ]:
from tensorflow import keras
model_l = keras.models.load_model('/content/drive/MyDrive/MODELS/VGG16.h5')

In [ ]:
import numpy as pd 
import cv2

upload_image=r'/content/Final IMG/test/non caries/116.png'
image=cv2.imread(upload_image)
converted_img = np.array(image)
img = cv2.resize(converted_img, dsize=(256,256))
img_reshape = np.reshape(img,[1,256,256,3])
y_predict = np.argmax(model_l.predict(img_reshape), axis=1)
# np.argmax(model_l.predict())
y_predict

1/1 [==============================] - 0s 16ms/step


array([0])

In [ ]:
model_l.predict(img_reshape)

1/1 [==============================] - 0s 16ms/step


array([[0.65555775, 0.34444225]], dtype=float32)

In [ ]:
lb.classes_

array(['caries', 'non caries'], dtype='<U10')